# Enhanced Fuel Consumption Prediction Model
## Complete Machine Learning Pipeline with Model Comparison and PKL Output

This notebook provides a comprehensive analysis of fuel consumption data from 2000-2022, including:
- Data exploration and visualization
- Multiple model training and comparison
- Best model selection and PKL file output
- Feature importance analysis
- Enhanced prediction capabilities


## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Configuration and Data Loading

In [ ]:
# Configuration
DATA_PATH = 'Fuel_Consumption_2000-2022.csv'
MODEL_OUTPUT_DIR = 'model'

# Define input and output features
INPUT_FEATURES = [
    'MAKE', 'MODEL', 'VEHICLE CLASS', 
    'ENGINE SIZE', 'CYLINDERS', 'TRANSMISSION', 'FUEL'
]

OUTPUT_FEATURES = ['COMB (L/100 km)', 'HWY (L/100 km)', 'EMISSIONS']

print(f"Data path: {DATA_PATH}")
print(f"Input features: {INPUT_FEATURES}")
print(f"Output features: {OUTPUT_FEATURES}")
print(f"Model output directory: {MODEL_OUTPUT_DIR}")

In [ ]:
def load_data(filepath):
    """Load and validate data from CSV file."""
    print(f"Loading data from {filepath}...")
    try:
        df = pd.read_csv(filepath)
        print(f"✓ Data loaded successfully. Shape: {df.shape}")
        print(f"✓ Columns available: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return None

# Load the dataset
df = load_data(DATA_PATH)

if df is not None:
    print(f"\nDataset overview:")
    print(df.head())

## 3. Data Exploration and Analysis

In [ ]:
def explore_basic_info(df):
    """Display basic information about the dataset."""
    print("=" * 60)
    print("BASIC DATASET INFORMATION")
    print("=" * 60)
    
    print(f"Dataset Shape: {df.shape}")
    print(f"Memory Usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
    
    print("\nData Types:")
    print(df.dtypes)
    
    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing Percentage': missing_pct
    })[missing > 0]
    
    if len(missing_df) > 0:
        print(missing_df)
    else:
        print("No missing values found!")
    
    print("\nBasic Statistics for Numerical Columns:")
    print(df.describe())

if df is not None:
    explore_basic_info(df)

In [ ]:
def create_data_visualizations(df):
    """Create comprehensive data visualizations."""
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Distribution of fuel consumption
    plt.subplot(3, 3, 1)
    plt.hist(df['COMB (L/100 km)'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Distribution of Combined Fuel Consumption')
    plt.xlabel('L/100 km')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # 2. Distribution of highway fuel consumption
    plt.subplot(3, 3, 2)
    plt.hist(df['HWY (L/100 km)'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
    plt.title('Distribution of Highway Fuel Consumption')
    plt.xlabel('L/100 km')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # 3. Distribution of emissions
    plt.subplot(3, 3, 3)
    plt.hist(df['EMISSIONS'], bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
    plt.title('Distribution of CO2 Emissions')
    plt.xlabel('g/km')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # 4. Fuel type distribution
    plt.subplot(3, 3, 4)
    fuel_counts = df['FUEL'].value_counts()
    plt.pie(fuel_counts.values, labels=fuel_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title('Fuel Type Distribution')
    
    # 5. Vehicle class distribution (top 10)
    plt.subplot(3, 3, 5)
    vc_counts = df['VEHICLE CLASS'].value_counts().head(10)
    plt.barh(range(len(vc_counts)), vc_counts.values)
    plt.yticks(range(len(vc_counts)), vc_counts.index)
    plt.title('Top 10 Vehicle Classes')
    plt.xlabel('Count')
    plt.grid(True, alpha=0.3)
    
    # 6. Engine size vs Combined consumption scatter
    plt.subplot(3, 3, 6)
    plt.scatter(df['ENGINE SIZE'], df['COMB (L/100 km)'], alpha=0.5, s=1)
    plt.xlabel('Engine Size (L)')
    plt.ylabel('Combined Consumption (L/100 km)')
    plt.title('Engine Size vs Fuel Consumption')
    plt.grid(True, alpha=0.3)
    
    # 7. Cylinders vs Combined consumption box plot
    plt.subplot(3, 3, 7)
    df_sample = df.sample(n=min(10000, len(df)))  # Sample for performance
    sns.boxplot(data=df_sample, x='CYLINDERS', y='COMB (L/100 km)')
    plt.title('Cylinders vs Fuel Consumption')
    plt.xticks(rotation=45)
    
    # 8. Year trend
    plt.subplot(3, 3, 8)
    yearly_avg = df.groupby('YEAR')['COMB (L/100 km)'].mean()
    plt.plot(yearly_avg.index, yearly_avg.values, marker='o', linewidth=2, markersize=6)
    plt.title('Average Fuel Consumption Trend Over Years')
    plt.xlabel('Year')
    plt.ylabel('Average Combined Consumption (L/100 km)')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    
    # 9. Make distribution (top 10)
    plt.subplot(3, 3, 9)
    make_counts = df['MAKE'].value_counts().head(10)
    plt.barh(range(len(make_counts)), make_counts.values)
    plt.yticks(range(len(make_counts)), make_counts.index)
    plt.title('Top 10 Vehicle Makes')
    plt.xlabel('Count')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

if df is not None:
    create_data_visualizations(df)

## 4. Correlation Analysis

In [ ]:
def analyze_correlations(df):
    """Analyze correlations between numerical variables."""
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    correlation_matrix = df[numerical_cols].corr()
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(correlation_matrix, 
                annot=True, 
                cmap='RdBu_r', 
                center=0, 
                square=True,
                fmt='.3f',
                cbar_kws={'shrink': 0.8})
    plt.title('Correlation Matrix of Numerical Features', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Show strongest correlations with target variables
    target_correlations = correlation_matrix[OUTPUT_FEATURES].drop(OUTPUT_FEATURES)
    print("\nStrongest correlations with target variables:")
    for target in OUTPUT_FEATURES:
        print(f"\n{target}:")
        corr_sorted = target_correlations[target].abs().sort_values(ascending=False)
        for feature, corr in corr_sorted.head(5).items():
            print(f"  {feature}: {target_correlations[target][feature]:.3f}")

if df is not None:
    analyze_correlations(df)

## 5. Data Preprocessing

In [ ]:
def preprocess_data(df, input_features, output_features):
    """Comprehensive data preprocessing pipeline."""
    print("=" * 60)
    print("DATA PREPROCESSING")
    print("=" * 60)
    
    # Select relevant columns and create copy
    all_features = input_features + output_features
    data = df[all_features].copy()
    
    print(f"Initial data shape: {data.shape}")
    
    # Handle missing values
    missing_before = data.isnull().sum().sum()
    if missing_before > 0:
        print(f"Missing values found: {missing_before}")
        data_clean = data.dropna()
        print(f"After removing missing values: {data_clean.shape}")
    else:
        data_clean = data
        print("No missing values to handle")
    
    # Separate features and targets
    X = data_clean[input_features]
    y = data_clean[output_features]
    
    # Identify feature types
    categorical_features = X.select_dtypes(include=['object']).columns.tolist()
    numerical_features = X.select_dtypes(include=['number']).columns.tolist()
    
    print(f"\nFeature analysis:")
    print(f"  Categorical features ({len(categorical_features)}): {categorical_features}")
    print(f"  Numerical features ({len(numerical_features)}): {numerical_features}")
    
    # Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
        ],
        remainder='passthrough'  # Keep numerical features as is
    )
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=None
    )
    
    print(f"\nData split:")
    print(f"  Training set: {X_train.shape[0]} samples")
    print(f"  Testing set: {X_test.shape[0]} samples")
    print(f"  Split ratio: {X_train.shape[0]/(X_train.shape[0]+X_test.shape[0]):.1%} train, {X_test.shape[0]/(X_train.shape[0]+X_test.shape[0]):.1%} test")
    
    return X_train, X_test, y_train, y_test, preprocessor, categorical_features, numerical_features

# Preprocess the data
if df is not None:
    X_train, X_test, y_train, y_test, preprocessor, categorical_features, numerical_features = preprocess_data(
        df, INPUT_FEATURES, OUTPUT_FEATURES
    )
    
    print("\nPreprocessing completed successfully!")
    print(f"Ready to train models with {X_train.shape[1]} input features")

## 6. Model Training and Comparison

In [ ]:
def define_models():
    """Define multiple models for comparison."""
    models = {
        'Random Forest': MultiOutputRegressor(RandomForestRegressor(
            n_estimators=200,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )),
        
        'Gradient Boosting': MultiOutputRegressor(GradientBoostingRegressor(
            n_estimators=100,
            max_depth=8,
            learning_rate=0.1,
            random_state=42
        )),
        
        'Decision Tree': MultiOutputRegressor(DecisionTreeRegressor(
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=4,
            random_state=42
        )),
        
        'Linear Regression': MultiOutputRegressor(LinearRegression()),
        
        'Ridge Regression': MultiOutputRegressor(Ridge(
            alpha=1.0,
            random_state=42
        )),
        
        'Neural Network': MultiOutputRegressor(MLPRegressor(
            hidden_layer_sizes=(100, 50, 25),
            max_iter=500,
            random_state=42,
            early_stopping=True,
            validation_fraction=0.1,
            alpha=0.001
        ))
    }
    
    return models

models = define_models()
print(f"Defined {len(models)} models for comparison:")
for name in models.keys():
    print(f"  - {name}")

In [ ]:
def train_and_evaluate_models(models, X_train, X_test, y_train, y_test, preprocessor, output_features):
    """Train and evaluate multiple models."""
    print("=" * 60)
    print("MODEL TRAINING AND EVALUATION")
    print("=" * 60)
    
    trained_models = {}
    all_metrics = {}
    training_times = {}
    
    for name, model in models.items():
        print(f"\n🔄 Training {name}...")
        
        # Create pipeline
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        try:
            # Train model with timing
            import time
            start_time = time.time()
            pipeline.fit(X_train, y_train)
            training_time = time.time() - start_time
            training_times[name] = training_time
            
            # Make predictions
            y_pred = pipeline.predict(X_test)
            
            # Calculate metrics for each target
            model_metrics = {}
            total_r2, total_rmse, total_mae = 0, 0, 0
            
            for i, target in enumerate(output_features):
                y_true = y_test[target]
                y_pred_target = y_pred[:, i] if y_pred.ndim > 1 else y_pred
                
                rmse = np.sqrt(mean_squared_error(y_true, y_pred_target))
                mae = mean_absolute_error(y_true, y_pred_target)
                r2 = r2_score(y_true, y_pred_target)
                
                model_metrics[target] = {
                    'rmse': rmse,
                    'mae': mae,
                    'r2': r2
                }
                
                total_r2 += r2
                total_rmse += rmse
                total_mae += mae
            
            # Calculate averages
            n_targets = len(output_features)
            model_metrics['average'] = {
                'r2': total_r2 / n_targets,
                'rmse': total_rmse / n_targets,
                'mae': total_mae / n_targets
            }
            
            trained_models[name] = pipeline
            all_metrics[name] = model_metrics
            
            # Display results
            avg_r2 = model_metrics['average']['r2']
            avg_rmse = model_metrics['average']['rmse']
            print(f"✓ {name} completed in {training_time:.2f}s")
            print(f"  Average R²: {avg_r2:.4f}")
            print(f"  Average RMSE: {avg_rmse:.4f}")
            
        except Exception as e:
            print(f"✗ Error training {name}: {str(e)}")
            continue
    
    return trained_models, all_metrics, training_times

# Train all models
if 'X_train' in locals():
    trained_models, all_metrics, training_times = train_and_evaluate_models(
        models, X_train, X_test, y_train, y_test, preprocessor, OUTPUT_FEATURES
    )
    print("\n🎉 Model training completed!")

## 7. Model Comparison and Best Model Selection

In [ ]:
def compare_and_visualize_models(all_metrics, training_times):
    """Compare models and visualize performance."""
    print("=" * 60)
    print("MODEL COMPARISON RESULTS")
    print("=" * 60)
    
    # Create comparison DataFrame
    comparison_data = []
    for name, metrics in all_metrics.items():
        comparison_data.append({
            'Model': name,
            'Avg_R2': metrics['average']['r2'],
            'Avg_RMSE': metrics['average']['rmse'],
            'Avg_MAE': metrics['average']['mae'],
            'Training_Time': training_times.get(name, 0)
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('Avg_R2', ascending=False)
    
    print("\nModel Performance Summary:")
    print(comparison_df.round(4))
    
    # Find best model
    best_model_name = comparison_df.iloc[0]['Model']
    best_r2 = comparison_df.iloc[0]['Avg_R2']
    best_rmse = comparison_df.iloc[0]['Avg_RMSE']
    
    print(f"\n🏆 Best Performing Model: {best_model_name}")
    print(f"   Average R²: {best_r2:.4f}")
    print(f"   Average RMSE: {best_rmse:.4f}")
    
    # Visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # R² Score comparison
    bars1 = ax1.bar(comparison_df['Model'], comparison_df['Avg_R2'], 
                    color=['gold' if name == best_model_name else 'lightblue' 
                          for name in comparison_df['Model']])
    ax1.set_title('Model Comparison - R² Score', fontweight='bold')
    ax1.set_ylabel('R² Score')
    ax1.set_ylim(0, 1)
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, value in zip(bars1, comparison_df['Avg_R2']):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # RMSE comparison
    bars2 = ax2.bar(comparison_df['Model'], comparison_df['Avg_RMSE'],
                    color=['gold' if name == best_model_name else 'lightcoral' 
                          for name in comparison_df['Model']])
    ax2.set_title('Model Comparison - RMSE', fontweight='bold')
    ax2.set_ylabel('RMSE')
    ax2.tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, value in zip(bars2, comparison_df['Avg_RMSE']):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(comparison_df['Avg_RMSE'])*0.02, 
                f'{value:.2f}', ha='center', va='bottom', fontweight='bold')
    
    # Training time comparison
    bars3 = ax3.bar(comparison_df['Model'], comparison_df['Training_Time'],
                    color='lightgreen')
    ax3.set_title('Training Time Comparison', fontweight='bold')
    ax3.set_ylabel('Time (seconds)')
    ax3.tick_params(axis='x', rotation=45)
    
    # Performance vs Time scatter
    scatter = ax4.scatter(comparison_df['Training_Time'], comparison_df['Avg_R2'], 
                         s=100, alpha=0.7, c=['gold' if name == best_model_name else 'blue' 
                                              for name in comparison_df['Model']])
    ax4.set_xlabel('Training Time (seconds)')
    ax4.set_ylabel('R² Score')
    ax4.set_title('Performance vs Training Time', fontweight='bold')
    
    # Add model name labels
    for i, name in enumerate(comparison_df['Model']):
        ax4.annotate(name, (comparison_df.iloc[i]['Training_Time'], comparison_df.iloc[i]['Avg_R2']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    return best_model_name, comparison_df

if 'all_metrics' in locals():
    best_model_name, comparison_df = compare_and_visualize_models(all_metrics, training_times)
    best_model = trained_models[best_model_name]

## 8. Detailed Evaluation of Best Model

In [ ]:
def detailed_model_evaluation(model, X_test, y_test, model_name, output_features):
    """Perform detailed evaluation of the best model."""
    print(f"=" * 60)
    print(f"DETAILED EVALUATION - {model_name}")
    print(f"=" * 60)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Create detailed visualizations
    fig, axes = plt.subplots(2, len(output_features), figsize=(18, 12))
    if len(output_features) == 1:
        axes = axes.reshape(-1, 1)
    
    detailed_metrics = {}
    
    for i, target in enumerate(output_features):
        y_true = y_test[target]
        y_pred_target = y_pred[:, i] if y_pred.ndim > 1 else y_pred
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_true, y_pred_target))
        mae = mean_absolute_error(y_true, y_pred_target)
        r2 = r2_score(y_true, y_pred_target)
        
        # Calculate MAPE (Mean Absolute Percentage Error)
        mape = np.mean(np.abs((y_true - y_pred_target) / y_true)) * 100
        
        detailed_metrics[target] = {
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2,
            'MAPE': mape
        }
        
        print(f"\n{target} Metrics:")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE: {mae:.4f}")
        print(f"  R²: {r2:.4f}")
        print(f"  MAPE: {mape:.2f}%")
        
        # Actual vs Predicted scatter plot
        axes[0, i].scatter(y_true, y_pred_target, alpha=0.5, s=1)
        axes[0, i].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 
                       'r--', lw=2, label='Perfect Prediction')
        axes[0, i].set_xlabel(f'Actual {target}')
        axes[0, i].set_ylabel(f'Predicted {target}')
        axes[0, i].set_title(f'{target}\nR² = {r2:.4f}')
        axes[0, i].legend()
        axes[0, i].grid(True, alpha=0.3)
        
        # Residual plot
        residuals = y_true - y_pred_target
        axes[1, i].scatter(y_pred_target, residuals, alpha=0.5, s=1)
        axes[1, i].axhline(y=0, color='r', linestyle='--', linewidth=2)
        axes[1, i].set_xlabel(f'Predicted {target}')
        axes[1, i].set_ylabel('Residuals')
        axes[1, i].set_title(f'Residual Plot - {target}')
        axes[1, i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return detailed_metrics

if 'best_model' in locals():
    detailed_metrics = detailed_model_evaluation(
        best_model, X_test, y_test, best_model_name, OUTPUT_FEATURES
    )

## 9. Feature Importance Analysis

In [ ]:
def analyze_feature_importance(model, categorical_features, numerical_features, preprocessor, model_name):
    """Analyze and visualize feature importance for tree-based models."""
    print(f"=" * 60)
    print(f"FEATURE IMPORTANCE ANALYSIS - {model_name}")
    print(f"=" * 60)
    
    try:
        # Check if model supports feature importance
        if hasattr(model.named_steps['model'], 'estimators_'):
            # Get feature names after preprocessing
            ohe = preprocessor.named_transformers_['cat']
            cat_feature_names = ohe.get_feature_names_out(categorical_features)
            all_feature_names = np.concatenate([cat_feature_names, numerical_features])
            
            # Extract feature importances
            estimators = model.named_steps['model'].estimators_
            n_features = len(all_feature_names)
            total_importances = np.zeros(n_features)
            
            # Average importance across all estimators
            for estimator in estimators:
                if hasattr(estimator, 'feature_importances_'):
                    total_importances += estimator.feature_importances_
            
            avg_importances = total_importances / len(estimators)
            
            # Create importance DataFrame
            importance_df = pd.DataFrame({
                'Feature': all_feature_names,
                'Importance': avg_importances
            }).sort_values('Importance', ascending=False)
            
            print(f"\nTop 20 Most Important Features:")
            top_features = importance_df.head(20)
            print(top_features)
            
            # Visualize feature importance
            plt.figure(figsize=(14, 10))
            sns.barplot(data=top_features, y='Feature', x='Importance', palette='viridis')
            plt.title(f'Top 20 Feature Importance - {model_name}', fontsize=16, fontweight='bold')
            plt.xlabel('Importance Score', fontsize=12)
            plt.ylabel('Features', fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Group importance by original feature categories
            feature_groups = {}
            for feature, importance in zip(all_feature_names, avg_importances):
                # Determine original feature group
                original_feature = None
                for cat_feat in categorical_features:
                    if feature.startswith(f"{cat_feat}_"):
                        original_feature = cat_feat
                        break
                if original_feature is None and feature in numerical_features:
                    original_feature = feature
                
                if original_feature:
                    if original_feature not in feature_groups:
                        feature_groups[original_feature] = 0
                    feature_groups[original_feature] += importance
            
            # Plot grouped importance
            groups_df = pd.DataFrame(
                list(feature_groups.items()), 
                columns=['Feature_Group', 'Total_Importance']
            ).sort_values('Total_Importance', ascending=False)
            
            plt.figure(figsize=(12, 6))
            sns.barplot(data=groups_df, x='Feature_Group', y='Total_Importance', palette='Set2')
            plt.title(f'Feature Group Importance - {model_name}', fontsize=16, fontweight='bold')
            plt.xlabel('Feature Groups', fontsize=12)
            plt.ylabel('Total Importance Score', fontsize=12)
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            print(f"\nFeature Group Importance:")
            print(groups_df)
            
            return importance_df, groups_df
            
        else:
            print(f"Feature importance not available for {model_name}")
            return None, None
            
    except Exception as e:
        print(f"Error analyzing feature importance: {str(e)}")
        return None, None

if 'best_model' in locals() and 'categorical_features' in locals():
    importance_df, groups_df = analyze_feature_importance(
        best_model, categorical_features, numerical_features, preprocessor, best_model_name
    )

## 10. Save Best Model as PKL File

In [ ]:
def save_best_model_pkl(model, model_name, metrics, output_dir='model'):
    """Save the best model as a PKL file with metadata."""
    print(f"=" * 60)
    print(f"SAVING BEST MODEL TO PKL FILE")
    print(f"=" * 60)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Create model filename
    safe_model_name = model_name.lower().replace(" ", "_").replace("-", "_")
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f'best_fuel_model_{safe_model_name}_{timestamp}.pkl'
    model_path = os.path.join(output_dir, model_filename)
    
    # Prepare model package with metadata
    model_package = {
        'model': model,
        'model_name': model_name,
        'metrics': metrics,
        'input_features': INPUT_FEATURES,
        'output_features': OUTPUT_FEATURES,
        'timestamp': pd.Timestamp.now(),
        'sklearn_version': __import__('sklearn').__version__,
        'pandas_version': pd.__version__,
        'numpy_version': np.__version__
    }
    
    # Save the model
    joblib.dump(model_package, model_path, compress=3)
    
    # Verify file was created and get size
    if os.path.exists(model_path):
        file_size = os.path.getsize(model_path) / (1024 * 1024)  # Size in MB
        print(f"✓ Model successfully saved to: {model_path}")
        print(f"✓ File size: {file_size:.2f} MB")
        print(f"✓ Model type: {model_name}")
        print(f"✓ Average R²: {metrics['average']['r2']:.4f}")
        print(f"✓ Average RMSE: {metrics['average']['rmse']:.4f}")
        
        # Create a simple model info file
        info_filename = f'model_info_{safe_model_name}_{timestamp}.txt'
        info_path = os.path.join(output_dir, info_filename)
        
        with open(info_path, 'w') as f:
            f.write(f"Fuel Consumption Prediction Model\n")
            f.write(f"{'='*40}\n\n")
            f.write(f"Model Type: {model_name}\n")
            f.write(f"Created: {pd.Timestamp.now()}\n")
            f.write(f"File: {model_filename}\n\n")
            
            f.write(f"Performance Metrics:\n")
            f.write(f"Average R²: {metrics['average']['r2']:.4f}\n")
            f.write(f"Average RMSE: {metrics['average']['rmse']:.4f}\n")
            f.write(f"Average MAE: {metrics['average']['mae']:.4f}\n\n")
            
            f.write(f"Target Variables:\n")
            for target in OUTPUT_FEATURES:
                if target in metrics:
                    f.write(f"  {target}:\n")
                    f.write(f"    R²: {metrics[target]['r2']:.4f}\n")
                    f.write(f"    RMSE: {metrics[target]['rmse']:.4f}\n")
                    f.write(f"    MAE: {metrics[target]['mae']:.4f}\n")
            
            f.write(f"\nInput Features: {INPUT_FEATURES}\n")
            f.write(f"Output Features: {OUTPUT_FEATURES}\n")
        
        print(f"✓ Model info saved to: {info_path}")
        
        return model_path, info_path
    else:
        print(f"✗ Error: Failed to save model to {model_path}")
        return None, None

if 'best_model' in locals() and 'all_metrics' in locals():
    model_path, info_path = save_best_model_pkl(
        best_model, best_model_name, all_metrics[best_model_name], MODEL_OUTPUT_DIR
    )

## 11. Enhanced Prediction Function and Demo

In [ ]:
def create_prediction_function(model):
    """Create an enhanced prediction function."""
    
    def predict_fuel_consumption(vehicle_specs):
        """
        Predict fuel consumption and emissions for a vehicle.
        
        Args:
            vehicle_specs (dict): Dictionary with vehicle specifications
                Required keys: 'MAKE', 'MODEL', 'VEHICLE CLASS', 'ENGINE SIZE', 
                              'CYLINDERS', 'TRANSMISSION', 'FUEL'
        
        Returns:
            dict: Comprehensive predictions and analysis
        """
        # Convert to DataFrame
        input_df = pd.DataFrame([vehicle_specs])
        
        # Make predictions
        predictions = model.predict(input_df)[0]
        
        # Extract predictions
        combined_consumption = predictions[0]
        highway_consumption = predictions[1]
        emissions = predictions[2]
        
        # Calculate additional metrics
        city_consumption = combined_consumption * 1.25  # Estimated city consumption
        
        # Efficiency rating
        def get_efficiency_rating(consumption):
            if consumption < 6: return "Excellent"
            elif consumption < 8: return "Good"
            elif consumption < 10: return "Average"
            elif consumption < 12: return "Below Average"
            else: return "Poor"
        
        # Annual cost estimation (assuming 15,000 km/year, $1.50/L)
        annual_km = 15000
        fuel_price = 1.50
        annual_liters = (combined_consumption * annual_km) / 100
        annual_cost = annual_liters * fuel_price
        
        # Environmental impact
        annual_emissions_kg = (emissions * annual_km) / 1000  # Convert g to kg
        
        return {
            'Predictions': {
                'Combined_Consumption_L_100km': round(combined_consumption, 2),
                'Highway_Consumption_L_100km': round(highway_consumption, 2),
                'City_Consumption_L_100km': round(city_consumption, 2),
                'CO2_Emissions_g_km': round(emissions, 2)
            },
            'Analysis': {
                'Efficiency_Rating': get_efficiency_rating(combined_consumption),
                'Annual_Fuel_Cost_CAD': round(annual_cost, 2),
                'Annual_CO2_Emissions_kg': round(annual_emissions_kg, 2),
                'Fuel_Efficiency_MPG': round(235.214583 / combined_consumption, 1)  # L/100km to MPG conversion
            },
            'Vehicle_Info': vehicle_specs
        }
    
    return predict_fuel_consumption

if 'best_model' in locals():
    predict_fuel = create_prediction_function(best_model)
    print("Enhanced prediction function created successfully!")

In [ ]:
def demonstration_predictions():
    """Demonstrate the prediction function with example vehicles."""
    print("=" * 60)
    print("PREDICTION DEMONSTRATIONS")
    print("=" * 60)
    
    # Example vehicles for demonstration
    example_vehicles = [
        {
            'name': 'Compact Car',
            'specs': {
                'MAKE': 'TOYOTA',
                'MODEL': 'COROLLA',
                'VEHICLE CLASS': 'COMPACT',
                'ENGINE SIZE': 1.8,
                'CYLINDERS': 4,
                'TRANSMISSION': 'AS',
                'FUEL': 'X'
            }
        },
        {
            'name': 'Mid-size SUV',
            'specs': {
                'MAKE': 'BMW',
                'MODEL': 'X5',
                'VEHICLE CLASS': 'SUV: SMALL',
                'ENGINE SIZE': 3.0,
                'CYLINDERS': 6,
                'TRANSMISSION': 'A8',
                'FUEL': 'Z'
            }
        },
        {
            'name': 'Luxury Sedan',
            'specs': {
                'MAKE': 'MERCEDES-BENZ',
                'MODEL': 'E-CLASS',
                'VEHICLE CLASS': 'MID-SIZE',
                'ENGINE SIZE': 2.0,
                'CYLINDERS': 4,
                'TRANSMISSION': 'A9',
                'FUEL': 'Z'
            }
        },
        {
            'name': 'Pickup Truck',
            'specs': {
                'MAKE': 'FORD',
                'MODEL': 'F-150',
                'VEHICLE CLASS': 'PICKUP TRUCK: STANDARD',
                'ENGINE SIZE': 5.0,
                'CYLINDERS': 8,
                'TRANSMISSION': 'A10',
                'FUEL': 'Z'
            }
        }
    ]
    
    for vehicle in example_vehicles:
        print(f"\n🚗 {vehicle['name']} Prediction:")
        print("-" * 40)
        
        try:
            result = predict_fuel(vehicle['specs'])
            
            print(f"Vehicle: {result['Vehicle_Info']['MAKE']} {result['Vehicle_Info']['MODEL']}")
            print(f"Class: {result['Vehicle_Info']['VEHICLE CLASS']}")
            print(f"Engine: {result['Vehicle_Info']['ENGINE SIZE']}L, {result['Vehicle_Info']['CYLINDERS']} cylinders")
            
            print("\nPredicted Consumption:")
            for key, value in result['Predictions'].items():
                print(f"  {key.replace('_', ' ')}: {value}")
            
            print("\nAnalysis:")
            for key, value in result['Analysis'].items():
                print(f"  {key.replace('_', ' ')}: {value}")
                
        except Exception as e:
            print(f"Error making prediction: {str(e)}")
    
    print("\n" + "=" * 60)
    print("Demonstration completed!")
    print("=" * 60)

if 'predict_fuel' in locals():
    demonstration_predictions()

## 12. Summary and Conclusion

In [ ]:
def create_final_summary():
    """Create a comprehensive summary of the analysis."""
    print("=" * 80)
    print("COMPREHENSIVE ANALYSIS SUMMARY")
    print("=" * 80)
    
    if 'df' in locals() and df is not None:
        print(f"\n📊 Dataset Information:")
        print(f"   • Total records: {len(df):,}")
        print(f"   • Years covered: {df['YEAR'].min()} - {df['YEAR'].max()}")
        print(f"   • Unique makes: {df['MAKE'].nunique()}")
        print(f"   • Unique models: {df['MODEL'].nunique()}")
    
    if 'best_model_name' in locals():
        print(f"\n🏆 Best Model: {best_model_name}")
        if 'all_metrics' in locals() and best_model_name in all_metrics:
            metrics = all_metrics[best_model_name]['average']
            print(f"   • Average R² Score: {metrics['r2']:.4f}")
            print(f"   • Average RMSE: {metrics['rmse']:.4f}")
            print(f"   • Average MAE: {metrics['mae']:.4f}")
    
    if 'model_path' in locals() and model_path:
        print(f"\n💾 Model Saved:")
        print(f"   • File: {os.path.basename(model_path)}")
        print(f"   • Location: {os.path.dirname(model_path)}")
        if os.path.exists(model_path):
            size_mb = os.path.getsize(model_path) / (1024 * 1024)
            print(f"   • Size: {size_mb:.2f} MB")
    
    print(f"\n🎯 Model Capabilities:")
    print(f"   • Predicts combined fuel consumption (L/100km)")
    print(f"   • Predicts highway fuel consumption (L/100km)")
    print(f"   • Predicts CO2 emissions (g/km)")
    print(f"   • Provides efficiency ratings")
    print(f"   • Estimates annual costs and environmental impact")
    
    if 'importance_df' in locals() and importance_df is not None:
        print(f"\n🔍 Key Insights:")
        top_feature = importance_df.iloc[0]['Feature']
        print(f"   • Most important feature: {top_feature}")
        if 'groups_df' in locals() and groups_df is not None:
            top_group = groups_df.iloc[0]['Feature_Group']
            print(f"   • Most important feature group: {top_group}")
    
    print(f"\n📈 Model Performance:")
    if 'comparison_df' in locals():
        print(f"   • Total models compared: {len(comparison_df)}")
        best_performance = comparison_df.iloc[0]
        print(f"   • Best model accuracy: {best_performance['Avg_R2']:.1%}")
        print(f"   • Training time: {best_performance['Training_Time']:.2f} seconds")
    
    print(f"\n🚀 Usage Instructions:")
    print(f"   1. Load the model: model = joblib.load('{os.path.basename(model_path) if 'model_path' in locals() else 'model.pkl'}')")
    print(f"   2. Use the prediction function with vehicle specifications")
    print(f"   3. Required inputs: MAKE, MODEL, VEHICLE CLASS, ENGINE SIZE, CYLINDERS, TRANSMISSION, FUEL")
    print(f"   4. Outputs: Fuel consumption predictions and comprehensive analysis")
    
    print("\n" + "=" * 80)
    print("✅ ANALYSIS COMPLETED SUCCESSFULLY!")
    print("📁 All outputs saved to the model directory")
    print("🎉 Ready for deployment and real-world predictions!")
    print("=" * 80)

create_final_summary()

## 13. Next Steps and Usage Guide

### Loading the Saved Model
```python
import joblib
import pandas as pd

# Load the model package
model_package = joblib.load('model/best_fuel_model_[timestamp].pkl')
model = model_package['model']
input_features = model_package['input_features']
output_features = model_package['output_features']
```

### Making Predictions
```python
# Example prediction
vehicle_data = {
    'MAKE': 'TOYOTA',
    'MODEL': 'CAMRY',
    'VEHICLE CLASS': 'MID-SIZE',
    'ENGINE SIZE': 2.5,
    'CYLINDERS': 4,
    'TRANSMISSION': 'A8',
    'FUEL': 'X'
}

input_df = pd.DataFrame([vehicle_data])
predictions = model.predict(input_df)
```

### Model Performance
The trained model provides highly accurate predictions for:
- **Combined Fuel Consumption** (L/100 km)
- **Highway Fuel Consumption** (L/100 km)  
- **CO2 Emissions** (g/km)

### Applications
- Vehicle purchasing decisions
- Fleet management optimization
- Environmental impact assessment
- Fuel cost budgeting
- Regulatory compliance reporting